In [ ]:
import numpy as np
import random

# Triangular Membership Function: Defines how fuzzy a pixel value is within certain ranges
def triangular_membership(x, a, b, c):
    if x < a or x > c:
        return 0
    elif x <= b:
        return (x - a) / (b - a) if b != a else 0
    else:
        return (c - x) / (c - b) if c != b else 0

# Fuzzify an Image: Fuzzifies an image by assigning fuzzy values for light, medium, and dark intensities
def fuzzify_image(image):
    light = np.zeros_like(image, dtype=float)
    medium = np.zeros_like(image, dtype=float)
    dark = np.zeros_like(image, dtype=float)

    # Loop over each pixel and fuzzify its value
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel_value = image[i, j]
            light[i, j] = triangular_membership(pixel_value, 0, 0, 85)
            medium[i, j] = triangular_membership(pixel_value, 85, 130, 175)
            dark[i, j] = triangular_membership(pixel_value, 175, 255, 255)

    # Stack the fuzzy values (light, medium, dark) to create a 3-channel fuzzified image
    return np.stack([light, medium, dark], axis=-1)

# Preprocess Dataset: Select a random subset of images and labels and fuzzify the images
def preprocess_dataset(x_data, y_data, num_samples=1000):
    selected_indices = random.sample(range(len(x_data)), num_samples)
    selected_images = x_data[selected_indices]
    selected_labels = y_data[selected_indices]

    # Fuzzify the images
    fuzzified_images = np.array([fuzzify_image(img) for img in selected_images])

    # One-hot encode the labels
    fuzzified_labels = np.zeros((len(selected_labels), 10))
    for i, label in enumerate(selected_labels):
        fuzzified_labels[i, label] = 1

    # Normalize fuzzified images
    fuzzified_images /= np.max(fuzzified_images)
    return fuzzified_images, fuzzified_labels

# Xavier Initialization: A method for initializing weights in neural networks
def xavier_init(shape):
    return np.random.randn(*shape) * np.sqrt(2 / sum(shape))

# Convolution Operation: Applies a convolution operation between the image and a kernel
def convolve2d(image, kernel):
    kernel_height, kernel_width = kernel.shape
    img_height, img_width = image.shape

    output_height = img_height - kernel_height + 1
    output_width = img_width - kernel_width + 1

    output = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            region = image[i:i + kernel_height, j:j + kernel_width]
            output[i, j] = np.sum(region * kernel)

    return output

# Max Pooling: Applies max pooling to reduce the spatial dimensions of the feature map
def max_pooling(image, pool_size=2):
    output_height = image.shape[0] // pool_size
    output_width = image.shape[1] // pool_size
    pooled = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            region = image[i * pool_size:(i + 1) * pool_size, j * pool_size:(j + 1) * pool_size]
            pooled[i, j] = np.max(region)
    return pooled

# ReLU Activation: Applies ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Softmax Activation: Converts logits into probabilities
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Numerical stability
    return exp_x / np.sum(exp_x)

# Cross-Entropy Loss: Computes the cross-entropy loss between predictions and actual labels
def cross_entropy(predictions, targets):
    return -np.sum(targets * np.log(predictions + 1e-9)) / len(targets)

# Forward Pass: Performs the forward pass of the neural network
def forward_pass(image, params):
    # Convolutional Layers
    conv1 = relu(convolve2d(image, params["conv1_kernel"]))
    pool1 = max_pooling(conv1)

    conv2 = relu(convolve2d(pool1, params["conv2_kernel"]))
    pool2 = max_pooling(conv2)

    # Flatten Layer: Converts the pooled output into a 1D vector
    flattened = pool2.flatten()

    # Fully Connected Layers
    fc1 = relu(np.dot(flattened, params["fc1_weights"]) + params["fc1_biases"])
    logits = np.dot(fc1, params["fc2_weights"]) + params["fc2_biases"]

    # Output Layer with Softmax
    output = softmax(logits)
    return output, fc1, flattened

# Backpropagation: Computes gradients for backpropagation
def backpropagation(output, label, fc1, flattened, params, gradients):
    d_logits = output - label  # Derivative of softmax with cross-entropy

    # Gradients for the second fully connected layer
    gradients["fc2_weights"] += np.outer(fc1, d_logits)
    gradients["fc2_biases"] += d_logits

    # Gradients for the first fully connected layer
    d_fc1 = np.dot(d_logits, params["fc2_weights"].T) * (fc1 > 0)  # ReLU derivative
    gradients["fc1_weights"] += np.outer(flattened, d_fc1)
    gradients["fc1_biases"] += d_fc1

    return gradients

# Update Parameters: Updates the network parameters using the computed gradients
def update_parameters(params, gradients, learning_rate):
    for key in gradients.keys():
        params[key] -= learning_rate * gradients[key]
    return params

# Load MNIST Dataset: Loads the MNIST dataset and normalizes the images
def load_mnist_data():
    from tensorflow.keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize images
    return x_train, y_train, x_test, y_test

# Initialize Parameters: Initializes network parameters
def initialize_parameters(input_size):
    params = {
        "conv1_kernel": xavier_init((3, 3)),
        "conv2_kernel": xavier_init((3, 3)),
        "fc1_weights": xavier_init((input_size, 128)),
        "fc1_biases": np.zeros(128),
        "fc2_weights": xavier_init((128, 10)),
        "fc2_biases": np.zeros(10),
    }
    return params

# Accuracy Calculation: Computes accuracy by comparing predicted and true labels
def calculate_accuracy(images, labels, params):
    correct = 0
    total = len(images)

    for img, label in zip(images, labels):
        output, _, _ = forward_pass(img[:, :, 0], params)
        predicted = np.argmax(output)
        true_label = np.argmax(label)
        if predicted == true_label:
            correct += 1

    return correct / total

# Training Loop: Trains the model using stochastic gradient descent
def train_model(images, labels, params, epochs=30, batch_size=32, learning_rate=0.0005):
    for epoch in range(epochs):
        indices = np.arange(len(images))
        np.random.shuffle(indices)
        images, labels = images[indices], labels[indices]

        for i in range(0, len(images), batch_size):
            batch_images = images[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]

            gradients = {key: np.zeros_like(value) for key, value in params.items()}

            for img, label in zip(batch_images, batch_labels):
                output, fc1, flattened = forward_pass(img[:, :, 0], params)
                gradients = backpropagation(output, label, fc1, flattened, params, gradients)

            # Update Parameters after processing the batch
            params = update_parameters(params, gradients, learning_rate)

        # Calculate and print accuracy after each epoch
        epoch_accuracy = calculate_accuracy(images, labels, params)
        print(f"Epoch {epoch+1}/{epochs}, Accuracy: {epoch_accuracy * 100:.2f}%")

    return params

# Main Block: Loads data, preprocesses it, initializes parameters, and trains the model
if __name__ == "__main__":
    # Load Dataset
    x_train, y_train, x_test, y_test = load_mnist_data()

    # Preprocess Training Data (Fuzzified images)
    fuzzified_train, fuzzified_labels = preprocess_dataset(x_train, y_train, 5000)

    # Determine Input Size for the Fully Connected Layer
    sample_img = fuzzified_train[0, :, :, 0]
    conv1 = convolve2d(sample_img, np.random.randn(3, 3))
    pool1 = max_pooling(conv1)
    conv2 = convolve2d(pool1, np.random.randn(3, 3))
    pool2 = max_pooling(conv2)
    input_size = pool2.flatten().size

    # Initialize Parameters (weights and biases)
    params = initialize_parameters(input_size)

    # Train the Model
    params = train_model(fuzzified_train, fuzzified_labels, params, epochs=30, batch_size=32)

    # Evaluate the Model on Test Data
    fuzzified_test, test_labels = preprocess_dataset(x_test, y_test, 1000)
    predictions = []
    for img in fuzzified_test:
        output, _, _ = forward_pass(img[:, :, 0], params)
        predictions.append(np.argmax(output))

    # Calculate final accuracy
    accuracy = np.mean(np.argmax(test_labels, axis=1) == predictions)
    print(f"Final Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/30, Accuracy: 61.84%
Epoch 2/30, Accuracy: 71.10%
Epoch 3/30, Accuracy: 72.92%
Epoch 4/30, Accuracy: 74.90%
Epoch 5/30, Accuracy: 76.22%
Epoch 6/30, Accuracy: 76.98%
Epoch 7/30, Accuracy: 77.46%
Epoch 8/30, Accuracy: 77.92%
Epoch 9/30, Accuracy: 78.02%
Epoch 10/30, Accuracy: 78.70%
Epoch 11/30, Accuracy: 78.72%
Epoch 12/30, Accuracy: 78.76%
Epoch 13/30, Accuracy: 79.46%
Epoch 14/30, Accuracy: 79.48%
Epoch 15/30, Accuracy: 79.42%
Epoch 16/30, Accuracy: 80.02%
Epoch 17/30, Accuracy: 79.92%
Epoch 18/30, Accuracy: 80.28%
Epoch 19/30, Accuracy: 80.36%
Epoch 20/30, Accuracy: 80.90%
Epoch 21/30, Accuracy: 81.02%
Epoch 22/30, Accuracy: 80.98%
Epoch 23/30, Accuracy: 81.50%
Epoch 24/30, Accuracy: 81.46%
Epoch 25/30, Accuracy: 81.56%
Epoch 26/30, Accuracy: 82.00%
Epoch 27/30, Accuracy: 82.54%
Epoch 28/30, Accuracy: 82.34%
Epoch 29/30, Accuracy: 82.76%
Epoch 30/30, Accuracy: 82.78%
Final Test Accuracy: 81.70%
